Q1. Write a Python program to scrape all available books from the website(https://books.toscrape.com/) Books to Scrape – a live site built for practicing scraping (safe,legal, no anti-bot). <br>For each book, extract the following details:<br>1. Title<br>2. Price<br>3. Availability (In stock / Out of stock)<br>4. Star Rating (One, Two, Three, Four, Five)

In [53]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
url = "https://books.toscrape.com/catalogue/"

bookList = []
#print(soup.find_all("article", class_="product_pod"))

#recursive function to scrap page
def scrape_page(page_url):
    res = req.get(page_url)
    print(f"Scraping {res.url}")
    soup = bs(res.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")

    for article in books:
        title = article.h3.a["title"]
        price = article.find("div", class_="product_price").find("p", class_="price_color").text.strip()
        availibility = article.find("div", class_="product_price").find("p", class_="instock availability").text.strip()=='In stock'
        rating = article.p["class"][1]

        
        d = dict()
        d["Title"] = title
        d["Price"]=price
        d["Availibility"]=availibility
        d["Rating"]=rating

        bookList.append(d)
        # print(title)
        # print(price)
        # print(availibility)
        # print(rating)

    #print(bookList)
    pager = soup.find("ul",class_="pager")
    next_page_link = next_page_link = pager.find("li", class_="next").find("a") if pager.find("li", class_="next") else None
    if(next_page_link is not None):
         next_page = next_page_link.get("href")
         next_page_url = url+next_page
         scrape_page(next_page_url)
    print("Scrapping completed successfully")

start_url = url+"page-1.html"

scrape_page(start_url)

pd.DataFrame(bookList).to_csv("books_data.csv",",")


Scraping https://books.toscrape.com/catalogue/page-1.html
Scraping https://books.toscrape.com/catalogue/page-2.html
Scraping https://books.toscrape.com/catalogue/page-3.html
Scraping https://books.toscrape.com/catalogue/page-4.html
Scraping https://books.toscrape.com/catalogue/page-5.html
Scraping https://books.toscrape.com/catalogue/page-6.html
Scraping https://books.toscrape.com/catalogue/page-7.html
Scraping https://books.toscrape.com/catalogue/page-8.html
Scraping https://books.toscrape.com/catalogue/page-9.html
Scraping https://books.toscrape.com/catalogue/page-10.html
Scraping https://books.toscrape.com/catalogue/page-11.html
Scraping https://books.toscrape.com/catalogue/page-12.html
Scraping https://books.toscrape.com/catalogue/page-13.html
Scraping https://books.toscrape.com/catalogue/page-14.html
Scraping https://books.toscrape.com/catalogue/page-15.html
Scraping https://books.toscrape.com/catalogue/page-16.html
Scraping https://books.toscrape.com/catalogue/page-17.html
Scrapi

C:\Users\dheer\AppData\Local\Temp\ipykernel_9576\3396827857.py:48: FutureWarning: Starting with pandas version 3.0 all arguments of to_csv except for the argument 'path_or_buf' will be keyword-only.
  pd.DataFrame(bookList).to_csv("books_data.csv",",")


Q2. Write a Python program to scrape the IMDB Top 250 Movies list(https://www.imdb.com/chart/top/) .<br> For each movie, extract the following details:<br>1. Rank (1–250)<br>2. Movie Title<br>3. Year of Release<br>4. IMDB Rating<br>Store the results in a Pandas DataFrame and export it to a CSV file named imdb_top250.csv.<br>(Note: Use Selenium/Playwright to scrape the required details from this website)

In [91]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://www.imdb.com/chart/top/")

# Find all movie elements
movie_container = driver.find_elements(By.CSS_SELECTOR, "div.sc-15ac7568-0.jQHOho.cli-children")

movie_list = []

# Iterate through the list and print each movie title
for container in movie_container:
    rank,name = container.find_element(By.CSS_SELECTOR, "h3").text.split(".",1) #split into two parts by first .(dot char)
    release_year = container.find_element(By.CSS_SELECTOR,"span.sc-15ac7568-7.cCsint.cli-title-metadata-item").text
    rating = container.find_element(By.CSS_SELECTOR, "span.ipc-rating-star--rating").text
    
    d = dict()
    d = {
        "Rank": rank.strip(),
        "Title":name.strip(),
        "Year of Releas": release_year,
        "IMDB Rating": rating
    }
    movie_list.append(d)
    #print(d)
driver.quit()
pd.DataFrame(movie_list).to_csv(path_or_buf="imdb_top250.csv", index=False)
print("Movie List saved successfully, imdb_top250.csv")

Movie List saved successfully, imdb_top250.csv


Q3. Write a Python program to scrape the weather information for top world cities from the given website (https://www.timeanddate.com/weather/) . For each city, extract the followingdetails:<br><br>1. City Name<br>2. Temperature<br>3. Weather Condition (e.g., Clear, Cloudy, Rainy, etc.)<br><br>Store the results in a Pandas DataFrame and export it to a CSV file named weather.csv.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome()
driver.get("https://www.timeanddate.com/weather/")

rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

city_temp_list = []


#extract city, temp, and condition
def extract_city_data(tds, col_start):
    try:
        city_name = tds[col_start].text.strip()
        if not city_name:  # skip if city not available
            return None

        condition = tds[col_start+2].find_element(By.CSS_SELECTOR, "img").get_attribute("title") \
                    if tds[col_start+2].find_elements(By.CSS_SELECTOR, "img") else "N/A"

        temperature = tds[col_start+3].text.strip() if tds[col_start+3].text.strip() else "N/A"

        return {
            "City": city_name,
            "Temperature": temperature,
            "Weather Condition": condition
        }
    except Exception:
        return None

for j in range(0,12,4):
    for row in rows:
        tds = row.find_elements(By.CSS_SELECTOR, "td")
        if len(tds) >= 4:
            d = extract_city_data(tds, j)
            if d:
                print(f"Saving {d}")
                city_temp_list.append(d)
driver.quit()

# Saving CSV files
pd.DataFrame(city_temp_list).to_csv("weather.csv", index=False)
print(f"{len(city_temp_list)} cities' temperatures saved successfully, as weather.csv")


Saving {'City': 'Accra', 'Temperature': '24 °C', 'Weather Condition': 'Scattered clouds. Mild.'}
Saving {'City': 'Addis Ababa', 'Temperature': '17 °C', 'Weather Condition': 'Passing clouds. Mild.'}
Saving {'City': 'Adelaide', 'Temperature': '7 °C', 'Weather Condition': 'Chilly.'}
Saving {'City': 'Algiers', 'Temperature': '28 °C', 'Weather Condition': 'Overcast. Warm.'}
Saving {'City': 'Almaty', 'Temperature': '15 °C', 'Weather Condition': 'Passing clouds. Refreshingly cool.'}
Saving {'City': 'Amman', 'Temperature': '25 °C', 'Weather Condition': 'Clear. Warm.'}
Saving {'City': 'Amsterdam *', 'Temperature': '21 °C', 'Weather Condition': 'Sunny. Mild.'}
Saving {'City': 'Anadyr', 'Temperature': '5 °C', 'Weather Condition': 'Overcast. Quite cool.'}
Saving {'City': 'Anchorage *', 'Temperature': '13 °C', 'Weather Condition': 'Partly sunny. Cool.'}
Saving {'City': 'Ankara', 'Temperature': '24 °C', 'Weather Condition': 'Passing clouds. Mild.'}
Saving {'City': 'Antananarivo', 'Temperature': '14 

In [151]:
for i in range(0,12,4):
    print(i)

0
4
8
